In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_packed_sequence, pack_padded_sequence

from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

In [2]:
# !python -m spacy download fr_core_news_sm
!python -m spacy download de_core_news_sm
!python -m spacy download en_core_web_sm

# spacy_fr = spacy.load('fr_core_news_sm')
spacy_de = spacy.load('de_core_news_sm')
spacy_en = spacy.load('en_core_web_sm')

✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')


In [4]:
# def tokenize_fr(text):
#     return [tok.text for tok in spacy_fr.tokenizer(text)]

def tokenize_de(text):
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [tok.text for tok in spacy_en.tokenizer(text)]

# SRC = Field(tokenize=tokenize_fr,
SRC = Field(tokenize=tokenize_de, 
        init_token='<sos>',
        eos_token='<eos>',
        pad_token='<pad>',
        unk_token='<unk>',
        lower=True)

TRG = Field(tokenize=tokenize_en, 
            init_token='<sos>',
            eos_token='<eos>', 
            pad_token='<pad>',
            unk_token='<unk>',
            lower=True)

train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), fields = (SRC, TRG))

SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device,
    # memory 효율성 / 연산 속도 효율성을 위한 batch 길이 맞춰주기
    # pack padded sequence - pad packed sequence 랑 같이 쓸 수 있음
    sort_within_batch=True, 
    sort_key=lambda x: len(x.src))

In [29]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()

        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        # bidirection 때문에 enc_hid_dim * 2
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        embedded = self.dropout(self.embedding(src))
        # output은 가장 마지막의 hidden state, 즉 기존 seq2seq의 context vector와 동일
        # hidden 은 차례대로 layer 1 - forward / layer 1 - backward / ... layer n - forward / layer n - backward 순
        outputs, hidden = self.rnn(embedded)
        
        # foward / backward hidden state를 같이 고려할 수 있도록 설계함
        # 논문에서는 backward만 넣었다고 되어 있는데?
        # backward 를 넣는 것이 문장 전체에 대한 정확도를 높여주지 않을까? / RNN은 앞의 결과를 순차적으로 넣어주기 때문에?
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        return outputs, hidden

In [30]:
# badanau 식 attention 
# self-attention 이랑은 조금 다름
# attention을 학습하는 align neural network 를 jointly 학습시킨다는 특징이 있음

class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        # bias 가 제외된 것. embedding 과 유사한 효과를 낼 수 있는 것으로 보임
        # 연산도 비싸다는데 왜 굳이 ..?
        # https://medium.com/@mamunurrahamanmamun/how-word-embeddings-are-different-than-fully-connected-layers-9484abd85807
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs):        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        # 현재 상태에서 decoder는 하나 밖에 없으므로 전체 문장 길이만큼 반복해서 attention을 계산할 수 있도록 dimension을 맞춰줌
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        # encoder의 마지막 출력, 문장 전체에 대한 context를 내포. 
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        # 이후 batch matrix multiple 해야해서 batch를 앞으로 빼줌
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        attention = self.v(energy).squeeze(2)

        # 확률의 형태로 산출된 attention score
        return F.softmax(attention, dim=1)

In [31]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        self.embedding = nn.Embedding(output_dim, emb_dim)
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
        input = input.unsqueeze(0)      
        embedded = self.dropout(self.embedding(input))
        # softmax를 거쳐서 만들어진 attention score
        a = self.attention(hidden, encoder_outputs)
        a = a.unsqueeze(1)
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        weighted = torch.bmm(a, encoder_outputs)
        weighted = weighted.permute(1, 0, 2)
        
        # decoder의 인풋과 weigth를 합쳐서 RNN의 input 값으로 취한다
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        return prediction, hidden.squeeze(0)

In [32]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):        
        src_length, batch_size = src.shape
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            outputs[t] = output
            
            # input update / teacher forcing or moodel prediction
            # teacher forcing 은 exposure bias를 야기할 수 있음: infer 시 성능 저하 가능성
            # 그럼에도 최근에는 대부분 teacher forcing을 100으로 사용하는 듯 하다
            input = trg[t] if random.random() < teacher_forcing_ratio else output.argmax(1)

        return outputs

In [33]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

TRG_PAD_IDX = TRG.vocab.stoi[TRG.pad_token]

model = Seq2Seq(enc, dec, device).to(device)

def init_weights(m):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            
model.apply(init_weights)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

optimizer = optim.Adam(model.parameters())
criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

The model has 20,518,917 trainable parameters


In [34]:
def train(model, iterator, optimizer, criterion, clip):
    model.train()
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        src = batch.src
        trg = batch.trg

        optimizer.zero_grad()
        
        output = model(src, trg)
        output_dim = output.shape[-1]     
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        loss = criterion(output, trg)
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)     
        optimizer.step()

        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

def evaluate(model, iterator, criterion):
    model.eval()
    epoch_loss = 0
    
    with torch.no_grad():
        for i, batch in enumerate(iterator):
            src = batch.src
            trg = batch.trg

            #turn off teacher forcing
            output = model(src, trg, 0)
            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)
            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    
    return elapsed_mins, elapsed_secs

In [35]:
N_EPOCHS = 30
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut3-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 0m 23s
	Train Loss: 5.051 | Train PPL: 156.214
	 Val. Loss: 4.815 |  Val. PPL: 123.383
Epoch: 02 | Time: 0m 23s
	Train Loss: 4.130 | Train PPL:  62.163
	 Val. Loss: 4.190 |  Val. PPL:  66.051
Epoch: 03 | Time: 0m 23s
	Train Loss: 3.381 | Train PPL:  29.410
	 Val. Loss: 3.614 |  Val. PPL:  37.122
Epoch: 04 | Time: 0m 23s
	Train Loss: 2.881 | Train PPL:  17.829
	 Val. Loss: 3.374 |  Val. PPL:  29.199
Epoch: 05 | Time: 0m 22s
	Train Loss: 2.499 | Train PPL:  12.172
	 Val. Loss: 3.286 |  Val. PPL:  26.729
Epoch: 06 | Time: 0m 23s
	Train Loss: 2.220 | Train PPL:   9.212
	 Val. Loss: 3.353 |  Val. PPL:  28.595
Epoch: 07 | Time: 0m 22s
	Train Loss: 1.977 | Train PPL:   7.221
	 Val. Loss: 3.164 |  Val. PPL:  23.667
Epoch: 08 | Time: 0m 22s
	Train Loss: 1.782 | Train PPL:   5.944
	 Val. Loss: 3.261 |  Val. PPL:  26.070
Epoch: 09 | Time: 0m 22s
	Train Loss: 1.640 | Train PPL:   5.154
	 Val. Loss: 3.276 |  Val. PPL:  26.480
Epoch: 10 | Time: 0m 22s
	Train Loss: 1.532 | Train PPL

In [36]:
model.load_state_dict(torch.load('tut3-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

| Test Loss: 3.163 | Test PPL:  23.637 |
